Installing dependencies

In [1]:
! pip3 install -e .

Obtaining file:///C:/Users/s1859154/Desktop/Uni%20courses/Reinforcement%20Learning/uoe-rl2023-coursework
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\s1859154\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\s1859154\\AppData\\Local\\Temp\\pip-install-_zwsos3a\\box2d-py_c91050063c0a473f8cc7c67d961fada3\\setup.py'"'"'; __file__='"'"'C:\\Users\\s1859154\\AppData\\Local\\Temp\\pip-install-_zwsos3a\\box2d-py_c91050063c0a473f8cc7c67d961fada3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\s1859154\AppData\Local\Temp\pip-wheel-hz2_vu40'
       cwd: C:\Users\s1859154\AppData\Local\Temp\pip-install-_zwsos3a\box2d-py_c91050063c0a473f8cc7c67d961fada3\
  Complete output (16 lines):
  Using setuptools (version 61.2.0).
  running bdist_wheel


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852315 sha256=2536a06196a949ac9e631df5a0fcb7b0f41f6d7d4037f1f42689049fe1ca5155
  Stored in directory: c:\users\s1859154\appdata\local\pip\cache\wheels\05\4d\6c\d0ef0db36695ce032fe20099e3149d8db85cf36656176ff745
  Running setup.py clean for box2d-py
Successfully built gym
Failed to build box2d-py
    Running


  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\Box2D
  copying library\Box2D\Box2D.py -> build\lib.win-amd64-3.9\Box2D
  copying library\Box2D\__init__.py -> build\lib.win-amd64-3.9\Box2D
  creating build\lib.win-amd64-3.9\Box2D\b2
  copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-3.9\Box2D\b2
  running build_ext
  building 'Box2D._Box2D' extension
  swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
  swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp Box2D\Box2D.i
  error: command 'swig.exe' failed: None
  ----------------------------------------
  ERROR: Failed building wheel for box2d-py
  DEPRECATION: box2d-py was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You 

# Q1 Dynamic Programming
- To find optimal policies for Markov Decision Processes (MDPs)
- Implement the Policy Iteration (PI) and Value Iteration (VI) algorithms (functions).
- For each algorithm, you can find the functions that you need to implement under Tasks below. 
- Read the code documentation to understand the input and required outputs of these functions. 
- We will mark your submission only based on the correctness of the outputs of these functions.

In [2]:
# from rl2023.exercise1.mdp_solver import ValueIteration, PolicyIteration
from rl2023.exercise1.mdp import MDP, Transition, State, Action
from rl2023 import constants
import numpy as np

In [3]:
constants.EX1_CONSTANTS

{'gamma': 0.8}

In [4]:
def calculate_state_values(policy, rewards, gamma=0.9, theta=0.01):
    """
  Args:
      policy (np.array): Policy giving the probability of taking each action from each state.
      rewards (np.array): Rewards corresponding to reaching the next state from each state (so r(s,s') here rather than the usual r(s,a)).
      gamma (float, optional): Discount factor. Defaults to 0.9.
      theta (float, optional): Minimum value change threshold to terminate iteration. Defaults to 0.01.

  Returns:
      values (np.array): State values for given policy and reward mappings.
  """    
    num_states, num_actions = policy.shape # 2, 3
    values = np.zeros((num_states+1)) # Include terminal state
    
    print(f'Beginning policy evaluation for given policy and MDP...\n')
    iteration = 1

    while True:
        print(f'Iteration: {iteration} \t Current Values: {values}')
        delta=0
        initial_values = values
        values = np.zeros_like(values)
        for state in range(num_states):
            for action in range(num_actions):
                # With probability 0.9, next state = action with corresponding reward. With probability 0.1, next state = state with no reward.
                next_state_probabilities = {action:0.9, state:0.1}
                # Often we leave this next state computation (transition dynamics) to the environment. But to use complete dynamic programming we must know 
                # the transition probablities.
                for next_state in next_state_probabilities.keys():
                  # Note the expectation here is over both the policy and next state probabilities (environment dynamics)
                    aa = policy[state][action]*next_state_probabilities[next_state] * (rewards[state][next_state] + gamma * initial_values[next_state])
                    values[state] += aa
#                     print(aa)

            delta = max(delta, abs(initial_values[state]-values[state]))

        if delta < theta:
            print(f'\nMax difference in state value from previous iteration = {delta} which is less than threshold {theta}. Policy Evaluation terminating...\n')
            break

        iteration+=1

    print(f'Final policy state values: {values}')
    return values

In [7]:
def calculate_greedy_policy(policy, rewards, values, valid_actions, gamma=0.9):
    """Improve policy (take greedy actions) with respect to rewards and state values.

  Args:
      policy (np.array): Policy giving the probability of taking each action from each state.
      rewards (np.array): Rewards corresponding to reaching the next state from each state (so r(s,s') here rather than the usual r(s,a)).
      values (np.array): State values evaluated for current policy.
      gamma (float, optional): Discount factor. Defaults to 0.9.

  Returns:
      policy (np.array): Improved (greedy) policy.
  """

    num_states, num_actions = policy.shape # 2, 3
    greedy_policy = np.zeros_like(policy)
    state_action_values = np.nan*policy
    for action in range(num_actions):
        for state in range(num_states):
            if valid_actions[state, action]==1:
                state_action_values[state][action] = 0.0
                # With probability 0.9, next state = action with corresponding reward. With probability 0.1, next state = state with no reward.
                next_state_probabilities = {action:0.9, state:0.1}
                # Often the environment handles this next state computation (transition dynamics). But to use complete dynamic programming we must know 
                # the transition probablities.
        for next_state in next_state_probabilities.keys():
          # Note the expectation is now only over next state probabilities (environment dynamics) since we want the value for each valid state and action.
            state_action_values[state][action] += next_state_probabilities[next_state]*(rewards[state][next_state]+gamma*values[next_state])
            
    
    greedy_action = np.nanargmax(state_action_values[state]) # Argmax ignoring invalid actions with nan value
  
    for action in range(num_actions):
        greedy_policy[state][action] = 1 if action == greedy_action else 0

    print(f'State action values with previous policy:\n {state_action_values}\n')
    print(f'Greedy policy after policy improvement:\n {greedy_policy}')
    return greedy_policy

In [9]:
# state values
valid_actions = np.array([[0,1,0],[1,0,1]])

policy = np.array([[0,1,0],[0.5,0,0.5]])

rewards = np.array([[0,0,0],[0,0,10]])
###############################################################

# algorithm
values = calculate_state_values(policy, rewards, gamma=0.9, theta=0.01)

policy = calculate_greedy_policy(policy, rewards, values, valid_actions, gamma=0.9)

# values = calculate_state_values(policy, rewards, gamma=0.9, theta=0.01)

# policy = calculate_greedy_policy(policy, rewards, values, valid_actions, gamma=0.9)

Beginning policy evaluation for given policy and MDP...

Iteration: 1 	 Current Values: [0. 0. 0.]
Iteration: 2 	 Current Values: [0.  4.5 0. ]
Iteration: 3 	 Current Values: [3.645 4.905 0.   ]
Iteration: 4 	 Current Values: [4.3011   6.417675 0.      ]
Iteration: 5 	 Current Values: [5.58541575 6.81953625 0.        ]
Iteration: 6 	 Current Values: [6.02651178 7.37585164 0.        ]
Iteration: 7 	 Current Values: [6.51682589 7.60456392 0.        ]
Iteration: 8 	 Current Values: [6.7462111  7.82372524 0.        ]
Iteration: 9 	 Current Values: [6.94437644 7.93635077 0.        ]
Iteration: 10 	 Current Values: [7.053438   8.02674403 0.        ]
Iteration: 11 	 Current Values: [7.13647208 8.07904935 0.        ]
Iteration: 12 	 Current Values: [7.18631246 8.11738564 0.        ]
Iteration: 13 	 Current Values: [7.22185049 8.14102126 0.        ]
Iteration: 14 	 Current Values: [7.24419376 8.15754136 0.        ]
Iteration: 15 	 Current Values: [7.25958594 8.1680772  0.        ]

Max differen

In [21]:
class PolicyIteration(MDPSolver):
    """MDP solver using the Policy Iteration algorithm
    """

    def _policy_eval(self): #, policy: np.ndarray) -> np.ndarray:
        """Computes one policy evaluation step

        **YOU MUST IMPLEMENT THIS FUNCTION FOR Q1**

        :param policy (np.ndarray of float with dim (num of states, num of actions)):
            A 2D NumPy array that encodes the policy.
            It is indexed as (STATE, ACTION) where policy[STATE, ACTION] has the probability of
            taking action 'ACTION' in state 'STATE'.
            REMEMBER: the sum of policy[STATE, :] should always be 1.0
            For deterministic policies the following holds for each state S:
            policy[S, BEST_ACTION] = 1.0
            policy[S, OTHER_ACTIONS] = 0
        :return (np.ndarray of float with dim (num of states)): 
            A 1D NumPy array that encodes the computed value function
            It is indexed as (State) where V[State] is the value of state 'State'
        """
        ### PUT YOUR CODE HERE ###
        # initilise V(s)
        theta = 0.01 # self.theta
#         policy = np.zeros([self.state_dim, self.action_dim])
#         policy = np.array([[0,0.5,0.5],[1,0,0],[1,0,0]])
#         policy = np.array([[0,0,1],[1,0,0],[0.5,0.5,0]])
        policy = np.array([[1,0,0],[0,1,0],[0.5,0,0.5]])
        num_states, num_actions = policy.shape # 2, 3
    
        V = np.zeros(self.state_dim)
#         values = np.zeros((num_states + 1)) # Include terminal state
        gamma = 0.9 #self.gamma # obtain gamma value
        
        rewards = self.mdp.R # access rewards 
        print(rewards)
        print(self.mdp.P) 

        
        print(f'Beginning policy evaluation for given policy and MDP...\n')
        iteration = 1

        while True:
            print(f'Iteration: {iteration} \t Current Values: {V}')
            delta = 0
            # set v = V
            initial_values = V
            # V(s) = values
            V = np.zeros_like(V)
            for s in range(num_states):
                for a in range(num_actions):
                    for ns in range(num_states):
                        next_state_probabilities = self.mdp.P[s, a, ns] # *** how to get te next state policy?
                      # Note the expectation here is over both the policy and next state probabilities (environment dynamics)
                        V[s] += policy[s][a]*next_state_probabilities*(rewards[s, a, ns] + gamma * initial_values[ns])
#                         print(policy[s][a], next_state_probabilities, rewards[s, a, ns] , initial_values[ns])

                delta = max(delta, abs(initial_values[s]-V[s]))
        
            if delta < theta:
                print(f'\nMax difference in state value from previous iteration = {delta} which is less than threshold {theta}. Policy Evaluation terminating...\n')
                break

            iteration+=1

            print(f'Final policy state values: {V}')
        
        
#         raise NotImplementedError("Needed for Q1")
        return np.array(V)

    def _policy_improvement(self) -> Tuple[np.ndarray, np.ndarray]:
        """Computes policy iteration until a stable policy is reached

        **YOU MUST IMPLEMENT THIS FUNCTION FOR Q1**

        Useful Variables (As with Value Iteration):
        1. `self.mpd` -- Gives access to the MDP.
        2. `self.mdp.R` -- 3D NumPy array with the rewards for each transition.
            E.g. the reward of transition [3] -2-> [4] (going from state 3 to state 4 with action
            2) can be accessed with `self.R[3, 2, 4]`
        3. `self.mdp.P` -- 3D NumPy array with transition probabilities.
            *REMEMBER*: the sum of (STATE, ACTION, :) should be 1.0 (all actions lead somewhere)
            E.g. the transition probability of transition [3] -2-> [4] (going from state 3 to
            state 4 with action 2) can be accessed with `self.P[3, 2, 4]`

        :return (Tuple[np.ndarray of float with dim (num of states, num of actions),
                       np.ndarray of float with dim (num of states)):
            Tuple of calculated policy and value function
        """
        policy = np.zeros([self.state_dim, self.action_dim])
        V = np.zeros([self.state_dim])
        ### PUT YOUR CODE HERE ###
        raise NotImplementedError("Needed for Q1")
        return policy, V

    def solve(self, theta: float = 1e-6) -> Tuple[np.ndarray, np.ndarray]:
        """Solves the MDP

        This function compiles the MDP and then calls the
        policy improvement function that the student must implement
        and returns the solution

        **DO NOT CHANGE THIS FUNCTION**

        :param theta (float, optional): stop threshold, 
        defaults to 1e-6
        :return (Tuple[np.ndarray of float with dim (num of states, num of actions),
                       np.ndarray of float with dim (num of states)]):
            Tuple of calculated policy and value function
        """
        self.mdp.ensure_compiled()
        self.theta = theta
        return self._policy_eval()

In [170]:
# # rock 0, rock 1, land
policy = np.array([[0,1,0],[0.5,0,0.5],[0,0,0]])
# # policy = policy.transpose()
# # rock 1, land, rock 0
# policy = np.array([[0,0.5,0.5],[0,0,0],[1,0,0]])

for s in range(3):
    for a in range(3):
        print("state = ", s, "action = ", a, "policy =", policy[s][a] )
policy

state =  0 action =  0 policy = 0.0
state =  0 action =  1 policy = 1.0
state =  0 action =  2 policy = 0.0
state =  1 action =  0 policy = 0.5
state =  1 action =  1 policy = 0.0
state =  1 action =  2 policy = 0.5
state =  2 action =  0 policy = 0.0
state =  2 action =  1 policy = 0.0
state =  2 action =  2 policy = 0.0


array([[0. , 1. , 0. ],
       [0.5, 0. , 0.5],
       [0. , 0. , 0. ]])

In [22]:
mdp = MDP()
mdp.add_transition(
    #         start action end prob reward
    Transition("rock0", "jump0", "rock0", 1, 0),
    Transition("rock0", "stay", "rock0", 1, 0),
    Transition("rock0", "jump1", "rock0", 0.1, 0),
    Transition("rock0", "jump1", "rock1", 0.9, 0),
    Transition("rock1", "jump0", "rock1", 0.1, 0),
    Transition("rock1", "jump0", "rock0", 0.9, 0),
    Transition("rock1", "jump1", "rock1", 0.1, 0),
    Transition("rock1", "jump1", "land", 0.9, 10),
    Transition("rock1", "stay", "rock1", 1, 0),
    Transition("land", "stay", "land", 1, 0),
    Transition("land", "jump0", "land", 1, 0),
    Transition("land", "jump1", "land", 1, 0),
)
# mdp.transitions
solver = PolicyIteration(mdp, CONSTANTS["gamma"])
policy = solver.solve()
print("---Policy Iteration---")
print("Policy:")
print(solver.decode_policy(policy))
# # print("Value Function")
# # print(valuefunc)
print(policy)

[[[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0. 10.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
[[[0.1 0.  0.9]
  [1.  0.  0. ]
  [1.  0.  0. ]]

 [[0.  1.  0. ]
  [0.  1.  0. ]
  [0.  1.  0. ]]

 [[0.  0.9 0.1]
  [0.  0.  1. ]
  [0.9 0.  0.1]]]
Beginning policy evaluation for given policy and MDP...

Iteration: 1 	 Current Values: [0. 0. 0.]
Final policy state values: [0.  0.  4.5]
Iteration: 2 	 Current Values: [0.  0.  4.5]
Final policy state values: [3.645 0.    4.905]
Iteration: 3 	 Current Values: [3.645 0.    4.905]
Final policy state values: [4.3011   0.       6.417675]
Iteration: 4 	 Current Values: [4.3011   0.       6.417675]
Final policy state values: [5.58541575 0.         6.81953625]
Iteration: 5 	 Current Values: [5.58541575 0.         6.81953625]
Final policy state values: [6.02651178 0.         7.37585164]
Iteration: 6 	 Current Values: [6.02651178 0.         7.37585164]
Final policy state values: [6.51682589 0.    

In [16]:
# if __name__ == "__main__":
#     mdp = MDP()
#     mdp.add_transition(
#         #         start action end prob reward
#         Transition("rock0", "jump0", "rock0", 1, 0),
#         Transition("rock0", "stay", "rock0", 1, 0),
#         Transition("rock0", "jump1", "rock0", 0.1, 0),
#         Transition("rock0", "jump1", "rock1", 0.9, 0),
#         Transition("rock1", "jump0", "rock1", 0.1, 0),
#         Transition("rock1", "jump0", "rock0", 0.9, 0),
#         Transition("rock1", "jump1", "rock1", 0.1, 0),
#         Transition("rock1", "jump1", "land", 0.9, 10),
#         Transition("rock1", "stay", "rock1", 1, 0),
#         Transition("land", "stay", "land", 1, 0),
#         Transition("land", "jump0", "land", 1, 0),
#         Transition("land", "jump1", "land", 1, 0),
#     )

#     solver = ValueIteration(mdp, CONSTANTS["gamma"])
#     policy, valuefunc = solver.solve()
#     print("---Value Iteration---")
#     print("Policy:")
#     print(solver.decode_policy(policy))
#     print("Value Function")
#     print(valuefunc)

#     solver = PolicyIteration(mdp, CONSTANTS["gamma"])
#     policy, valuefunc = solver.solve()
#     print("---Policy Iteration---")
#     print("Policy:")
#     print(solver.decode_policy(policy))
#     print("Value Function")
#     print(valuefunc)


In [11]:
from abc import ABC, abstractmethod
import numpy as np
from typing import List, Tuple, Dict, Optional, Hashable

from rl2023.constants import EX1_CONSTANTS as CONSTANTS
from rl2023.exercise1.mdp import MDP, Transition, State, Action

class MDPSolver(ABC):
    """Base class for MDP solvers

    **DO NOT CHANGE THIS CLASS**

    :attr mdp (MDP): MDP to solve
    :attr gamma (float): discount factor gamma to use
    :attr action_dim (int): number of actions in the MDP
    :attr state_dim (int): number of states in the MDP
    """

    def __init__(self, mdp: MDP, gamma: float):
        """Constructor of MDPSolver

        Initialises some variables from the MDP, namely the state and action dimension variables

        :param mdp (MDP): MDP to solve
        :param gamma (float): discount factor (gamma)
        """
        self.mdp: MDP = mdp
        self.gamma: float = gamma

        self.action_dim: int = len(self.mdp.actions)
        self.state_dim: int = len(self.mdp.states)

    def decode_policy(self, policy: Dict[int, np.ndarray]) -> Dict[State, Action]:
        """Generates greedy, deterministic policy dict

        Given a stochastic policy from state indeces to distribution over actions, the greedy,
        deterministic policy is generated choosing the action with highest probability

        :param policy (Dict[int, np.ndarray of float with dim (num of actions)]):
            stochastic policy assigning a distribution over actions to each state index
        :return (Dict[State, Action]): greedy, deterministic policy from states to actions
        """
        new_p = {}
        for state, state_idx in self.mdp._state_dict.items():
            new_p[state] = self.mdp.actions[np.argmax(policy[state_idx])]
        return new_p

    @abstractmethod
    def solve(self):
        """Solves the given MDP
        """
        ...